In [95]:
import getpass
import os
import re
from typing import Annotated, TypedDict, List

import sympy as sp
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

llm = ChatOpenAI(model='gpt-4o-mini-2024-07-18', temperature=0)

MAX_ITERATIONS = 3

In [96]:
# Define a sympy-based tool
@tool
def calculate(expression: str) -> str:
    """Calculate an arithmetic expression using sympy.

    Args:
        expression: arithmetic expression as a string (e.g., '3 + 4')

    Returns:
        The evaluated result as a string.
    """
    try:
        expr = sp.sympify(expression)
        evaluated = expr.evalf() if expr.is_number else expr
        return str(evaluated)
    except Exception as e:
        return f"Error: {e}"

# Define a LLM tool
@tool
def llm_tool(expression: str) -> str:
    """Use LLM like yourself to process the input string.
    This is useful for tasks that require reasoning or understanding of the context.
    For example, you can use it to provide explanations.

    Args:
        expression: input string to be processed by LLM

    Returns:
        The processed output as a string.
    """
    response = llm.invoke([HumanMessage(content=expression)])
    return response.content


# Augment the LLM with the sympy tool
tools = [calculate, llm_tool]
tools_by_name = {tool.name: tool for tool in tools}

In [97]:
actor_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an expert math problem solver.
1. Solve the problem carefully, showing all steps.
2. Reflect on your solution - identify missing steps or unnecessary parts.
3. If needed, suggest calculations that could verify your answer."""),
    MessagesPlaceholder(variable_name="messages"),
    ("user", """Reflect on the user's original question and the steps taken thus far.
Respond using the {function_name} function."""),
])

In [98]:
class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing in the answer.")
    superfluous: str = Field(description="Critique of what is unnecessary.")

class MathSolution(BaseModel):
    """Solution to a math problem with reflection."""
    answer: str = Field(description="The calculated answer to the math problem.")
    steps: str = Field(description="Step-by-step reasoning to reach the answer.")
    reflection: Reflection = Field(description="Self-critique of the solution.")
    verify_calculations: List[str] = Field(
        description="List of calculations that should be verified with the calculator tool.",
        default_factory=list
    )

class RevisedSolution(MathSolution):
    """Revised math solution with improvements."""
    improvements: str = Field(description="What was improved from the initial solution.")

class Responder:
    def __init__(self, runnable, validator):
        self.runnable = runnable
        self.validator = validator

    def __call__(self, state: dict):
        response = self.runnable.invoke({"messages": state["messages"]})
        try:
            self.validator.invoke(response)
            return {"messages": [response]}
        except Exception as e:
            return {"messages": [ToolMessage(
                content=f"Validation error: {str(e)}",
                tool_call_id=response.tool_calls[0]['id'] if hasattr(response, 'tool_calls') else "0",
            )]}

initial_chain = (
    actor_prompt_template.partial(function_name=MathSolution.__name__)
    | llm.bind_tools(tools=[MathSolution], tool_choice=MathSolution.__name__)
)
initial_validator = PydanticToolsParser(tools=[MathSolution])
first_responder = Responder(runnable=initial_chain, validator=initial_validator)

revise_instructions = """Revise your previous solution using the critique:
- Add any missing steps or explanations
- Remove unnecessary parts
- Ensure calculations are correct
- The final answer should be precise and clearly formatted as "Answer: <value>"
"""
revision_chain = (
    actor_prompt_template.partial(
        first_instruction=revise_instructions,
        function_name=RevisedSolution.__name__,
    )
    | llm.bind_tools(tools=[RevisedSolution], tool_choice=RevisedSolution.__name__)
)
revision_validator = PydanticToolsParser(tools=[RevisedSolution])
revisor = Responder(runnable=revision_chain, validator=revision_validator)

class ToolNode:
    def __init__(self, tools):
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, state: dict):
        messages = state["messages"]
        last_message = messages[-1]

        if not hasattr(last_message, "tool_calls"):
            return {"messages": messages}

        tool_outputs = []
        for tool_call in last_message.tool_calls:
            if tool_call["name"] not in self.tools_by_name:
                continue

            tool = self.tools_by_name[tool_call["name"]]
            try:
                output = tool.invoke(tool_call["args"])
                tool_outputs.append(
                    ToolMessage(
                        content=str(output),
                        name=tool_call["name"],
                        tool_call_id=tool_call["id"],
                    )
                )
            except Exception as e:
                tool_outputs.append(
                    ToolMessage(
                        content=f"Error executing tool: {str(e)}",
                        name=tool_call["name"],
                        tool_call_id=tool_call["id"],
                    )
                )

        return {"messages": messages + tool_outputs}

class State(TypedDict):
    messages: Annotated[list, add_messages]

def format_final_answer(state: State):
    """Extracts and formats the final answer"""
    messages = state["messages"]

    # Find the last AI message with content
    last_ai_message = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.content:
            last_ai_message = msg
            break

    if not last_ai_message:
        return {"messages": [HumanMessage(content="No answer found")]}

    # Extract just the numeric answer
    answer = None
    try:
        # Try to find a number in the content
        match = re.search(r'\$?(\d+(\.\d+)?)', last_ai_message.content)
        if match:
            answer = match.group(1)
    except:
        pass

    if not answer:
        answer = "Could not extract numeric answer"

    return {"messages": [HumanMessage(content=f"Answer: {answer}")]}

def event_loop(state: State):
    messages = state["messages"]
    # Count iterations based on AI messages with tool calls
    iterations = sum(1 for msg in messages if isinstance(msg, AIMessage) and hasattr(msg, "tool_calls"))
    return "revise" if iterations < MAX_ITERATIONS else END


In [99]:
workflow = StateGraph(State)
workflow.add_node("draft", first_responder)
workflow.add_node("tools", ToolNode(tools))
workflow.add_node("revise", revisor)
workflow.add_node("format_answer", format_final_answer)

workflow.add_edge(START, "draft")
workflow.add_edge("draft", "tools")
workflow.add_edge("tools", "revise")
workflow.add_conditional_edges(
    "revise",
    event_loop,
    {"revise": "tools", END: "format_answer"}
)
workflow.add_edge("format_answer", END)

agent = workflow.compile()

task = "James has a $1000 budget. He spends 30% on food, 15% on accommodation, 25% on entertainment. \
How much does he spend on coursework materials?"

events = agent.stream(
    {"messages": [HumanMessage(content=task)]},
    stream_mode="values",
)

for i, step in enumerate(events):
    print(f"\nStep {i}")
    if step["messages"]:
        step["messages"][-1].pretty_print()


Step 0
================================ Human Message =================================

James has a $1000 budget. He spends 30% on food, 15% on accommodation, 25% on entertainment. How much does he spend on coursework materials?

Step 1
================================== Ai Message ==================================
Tool Calls:
  MathSolution (call_iUvIcfO60gXDDXBPXuYx9DF1)
 Call ID: call_iUvIcfO60gXDDXBPXuYx9DF1
  Args:
    answer: $500
    steps: 1. Calculate the total percentage spent on food, accommodation, and entertainment:
   - Food: 30%
   - Accommodation: 15%
   - Entertainment: 25%
   - Total percentage = 30% + 15% + 25% = 70%

2. Calculate the amount spent on food, accommodation, and entertainment:
   - Food: 30% of $1000 = 0.30 * 1000 = $300
   - Accommodation: 15% of $1000 = 0.15 * 1000 = $150
   - Entertainment: 25% of $1000 = 0.25 * 1000 = $250

3. Calculate the total amount spent on these three categories:
   - Total spent = $300 + $150 + $250 = $700

4. Calculate the

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_iUvIcfO60gXDDXBPXuYx9DF1", 'type': 'invalid_request_error', 'param': 'messages.[3].role', 'code': None}}